# Collect LoL match frames and save select data to match_id.csv

## Methodology
The algorithm used is based on [the information here](https://riot-api-libraries.readthedocs.io/en/latest/collectingdata.htmlhttps://riot-api-libraries.readthedocs.io/en/latest/collectingdata.html)
1. Get a high ranking summoner by hand. <br>
2. Get the account ID of this summoner.<br>
3. Get the summoner's match history .<br>
4. Get a list of the participants in the summoner's match history
5. Append the participants in the queue 
Success! You have a ton of match IDs, from there, you can fetch their timelines or parse the matches however you want.

In [1]:
import cassiopeia as cass
from cassiopeia.core import Summoner, MatchHistory, Match
from cassiopeia import Queue, Patch
import random
from sortedcontainers import SortedList
import arrow
import pandas as pd
import json
from pathlib import Path

### Constants API Key, API Region, Output df columns

- API Key should be saved in `api_key.txt`
- expires in 24 hours, so make sure to renew
- API key has rate limit dependant on region

In [2]:
# Get API Key from txt file so as not to show it in notebook
API_KEY = ""
with open('api_key.txt', 'r') as file:
    API_KEY = file.read().replace('\n', '')
API_REGION = 'RU'
df_columns = ["time", "r_kills", 'b_kills', 'r_towers', 'b_towers', 'r_monsters', 'b_monsters', 'r_ward_kills', 'b_ward_kills']
START_PATCH = "11.1"     # Patch Released 15/12/2020
COLLECTION_LIMIT = 25000 # 25000 matches

### Match Collection Functions

In [5]:

def get_participants_ids(participants):
    """Obtain a list of participant IDs from participants"""
    ids = [p.id for p in participants]
    return ids


def create_row(timestamp,r_kill=0, b_kill=0, r_tower=0, b_tower=0, r_monst=0, b_monst=0,
              r_ward_kills=0, b_ward_kills=0):
    """Create row for dataframe. Returns a list of the data, with missing data zeroed"""
    return [timestamp,r_kill, b_kill, r_tower, b_tower, r_monst, b_monst,  r_ward_kills, b_ward_kills]

def parse_frames(frames):
    """Parse match frames. Returns list of parsed data
    
    Currently, only CHAMPION_KILL, BUILDING_KILL, ELITE_MONSTER_KILL and WARD_KILL are parsed """
    df = []
    for frame in frames:
        for event in frame.events:
            time_s = event.timestamp.seconds//60
            if(event.type == 'CHAMPION_KILL'):
                if(event.killer_id < 6):
                    #Blue team
                    df.append(create_row(timestamp=time_s, b_kill=1))
                else:
                    df.append(create_row(timestamp=time_s, r_kill=1))
            elif(event.type == 'BUILDING_KILL'):
                if(event.to_dict()['buildingType'] == 'TOWER_BUILDING'): #somehow event.buildingType not working
                    if(event.killer_id < 6):
                        #Blue team
                        df.append(create_row(timestamp=time_s, b_tower=1))#red tower destroyed!!
                    else:
                        df.append(create_row(timestamp=time_s, r_tower=1))#blue tower destroyed!!
            elif(event.type == 'ELITE_MONSTER_KILL'): 
                if(event.killer_id < 6):
                        #Blue team
                    df.append(create_row(timestamp=time_s, b_monst=1))#red tower destroyed!!
                else:
                    df.append(create_row(timestamp=time_s, r_monst=1))#blue tower destroyed!!
            elif(event.type == 'WARD_KILL'): 
                if(event.killer_id < 6):
                        #Blue team
                    df.append(create_row(timestamp=time_s, b_ward_kills=1))#red tower destroyed!!
                else:
                    df.append(create_row(timestamp=time_s, r_ward_kills=1))#blue tower destroyed!!
    return df

def match_make_save_dir(region):
    """ Make save directory of match by region by region """
    path = 'lol_matches/'+ region
    csv_path = 'lol_matches/'+ region + '/csv'
    raw_json_path = 'lol_matches/'+ region +'/raw_json'
    Path(csv_path).mkdir(parents=True, exist_ok=True)
    Path(raw_json_path).mkdir(parents=True, exist_ok=True)
    return path, csv_path, raw_json_path

def match_data_to_csv(match_id, data, blue_win, path, columns=df_columns):
    """ Save match data as csv """
    df = pd.DataFrame(data=data, columns = columns)
    df = df.groupby(by='time').sum().reset_index()
    for name ,col  in df.iteritems():
        if (name != 'time'):
            df[name] = col.cumsum()
    df['Match_ID'] = match_id
    df['Win'] = blue_win
    
    file_path = path +'/' + str(match_id) + '.csv'
    df.to_csv(file_path)
    return df

#def match_save_csv(df, match_id, path):
#    df.to_csv(path +'/' + str(match_id) + '.csv' )
    
def match_timeline_to_json(match, path):
    """ Save match timeline as json """
    
    json_data = match.timeline.to_json()
    file_path = path  + '/' + str(match.id) + '.json'
    
    with open(file_path, 'w+') as out_json:
        json.dump(json_data, out_json) 
        
def match_save_id(match_id, path):
    """ Save parsed match ids so that we dont repeat """
    
    with open(path+'/parsed.txt', "a+") as file:
        file.write(str(match_id) + '\n')
        
def get_parsed_match_ids(path):
    """ Read parsed match ids"""
    file = path + '/parsed.txt'
    parsed_ids = []
    with open(file) as f:
        for line in f:
            parsed_ids.append(int(line))
    return parsed_ids

In [6]:
def init_cass(api_key, region):
    """Initialize csseiopia API"""
    
    assert api_key != "", "Please set the API key in the api_key.txt file"
        
    cass.set_riot_api_key(api_key)  # This overrides the value set in your configuration/settings.
    cass.set_default_region(region)

def filter_match_history(summoner, patch):
    """Filter summoners match history by patch date"""
    #end_time = patch.end
    #if end_time is None:
    end_time = arrow.now()
    match_history = MatchHistory(summoner=summoner, queues={Queue.aram}, begin_time=patch.start, end_time=end_time)
    print("Sommoner {} match history length: \t{}" .format(summoner.name,len(match_history)))
    return match_history


def collect_matches(initial_summoner_name = "Little Mandiocaa", region = API_REGION, patch_str = START_PATCH):
    """ Main function to collect matches """
    
    summoner = Summoner(name=initial_summoner_name, region=region)
    patch = Patch.from_str(patch_str, region=region)

    unpulled_summoner_ids = SortedList([summoner.id])
    pulled_summoner_ids = SortedList()

    unpulled_match_ids = SortedList()
    pulled_match_ids = SortedList()
    root_path, csv_path, raw_json_path  = match_make_save_dir(region=region)
    collection_ctr = 0
    
    # Get list of previously parsed matches
    parsed_match_ids = get_parsed_match_ids(root_path)
    
    while unpulled_summoner_ids:
        # Get a random summoner from our list of unpulled summoners and pull their match history
        new_summoner_id = random.choice(unpulled_summoner_ids)
        new_summoner = Summoner(id=new_summoner_id, region=region)
        matches = filter_match_history(new_summoner, patch)
        unpulled_match_ids.update([match.id for match in matches if not match.id in parsed_match_ids])
        unpulled_summoner_ids.remove(new_summoner_id)
        pulled_summoner_ids.add(new_summoner_id)

        while unpulled_match_ids:
            # Get a random match from our list of matches
            new_match_id = random.choice(unpulled_match_ids)
            new_match = Match(id=new_match_id, region=region)
            for participant in new_match.participants:
                if ((participant.summoner.id not in pulled_summoner_ids) & 
                    (participant.summoner.id not in unpulled_summoner_ids)):
                        
                    unpulled_summoner_ids.add(participant.summoner.id)
            # The above lines will trigger the match to load its data by iterating over all the participants.
            # If you have a database in your datapipeline, the match will automatically be stored in it.
            unpulled_match_ids.remove(new_match_id)
            pulled_match_ids.add(new_match_id)
            
            # Parse the match frames
            data = parse_frames(new_match.timeline.frames)
            
            # Save csv
            match_data_to_csv(match_id=new_match.id, 
                              data=data, 
                              blue_win=int(new_match.blue_team.win),
                              path=csv_path)
            # Save raw match data
            match_timeline_to_json(new_match, raw_json_path)
            
            # Save the match id ie that we've been here :)
            match_save_id(new_match.id, root_path)
            
            collection_ctr = collection_ctr + 1
            if(collection_ctr%100 == 0):
                print("Number of collected matches: {}" .format(collection_ctr))
            if(collection_ctr > COLLECTION_LIMIT):
                return

## Start the match collection

In [ ]:
initial_summoner_name_ru = "Огненный шторм"
summoner_region_ru = "RU"

initial_summoner_name_br = "Little Mandiocaa"
summoner_region_br = "BR"

initial_summoner_name_na = "From Iron"
summoner_region_na = "NA"

initial_summoner_name_eune = "Necromartin"
summoner_region_eune = "EUNE"

In [ ]:
init_cass(API_KEY, "EUW")

In [ ]:
#collect_matches(initial_summoner_name = "Slaytеr" , region = summoner_region_ru, patch_str = "11.1")

In [ ]:
#collect_matches(initial_summoner_name = "wintradeaccount0" , region = summoner_region_ru, patch_str = "11.1")

In [ ]:
try:
    collect_matches(initial_summoner_name = "SCHWARZFAUST" , region = summoner_region_ru, patch_str = "11.1")
except:
    pass

In [22]:
try:
    collect_matches(initial_summoner_name = "JaxLord69" , region = summoner_region_ru, patch_str = "11.1")
except:
    pass

Making call: https://ru.api.riotgames.com/lol/summoner/v4/summoners/by-name/JaxLord69
Making call: https://ru.api.riotgames.com/lol/summoner/v4/summoners/-29DOOH_rG26LqRwV7LENHD3pRAMKrIC_GUc5gbTO4hitw
Making call: https://ru.api.riotgames.com/lol/match/v4/matchlists/by-account/zPBgBLcsvXhtXFqxuWVxcyk-UQHe2L9u2o2hIfJ9WLepf4k?beginIndex=0&endIndex=100&queue=450
Sommoner JaxLord69 match history length: 	37


In [20]:
try:
    collect_matches(initial_summoner_name = "JaxLord69" , region = summoner_region_na, patch_str = "11.1")
except:
    pass

Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/JaxLord69


In [21]:
try:
    collect_matches(initial_summoner_name = "FLY Josédeodo" , region = summoner_region_na, patch_str = "11.1")
except:
    pass


In [12]:
try:
    collect_matches(initial_summoner_name = initial_summoner_name_br, region = summoner_region_br, patch_str = "11.1")
except:
    pass

Making call: https://br1.api.riotgames.com/lol/summoner/v4/summoners/by-name/LittleMandiocaa


In [13]:
try:
    collect_matches(initial_summoner_name = "Reziw" , region = summoner_region_br, patch_str = "11.1")
except:
    pass

Making call: https://br1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Reziw


In [18]:
try:
    collect_matches(initial_summoner_name = "DR1PLEX" , region = "EUW", patch_str = "11.1")
except:
    pass

In [19]:
try:
    collect_matches(initial_summoner_name = "Meruem12" , region = "EUW", patch_str = "11.1")
except:
    pass